In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [ ]:
pd.set_option('display.max_rows', 500)

## FUNCTIONS

In [ ]:
def basic_clean(a_df):
    '''Performs initial cleaning of df '''
    
    a_df = a_df.drop_duplicates(keep="first")
        
    return a_df

In [ ]:
def all_caps(a_df, a_list):
    '''Renders string df columns in supplied list in all caps'''
    
    for col in a_list:
        a_df[col] = a_df[col].str.upper()
        
    return a_df

In [ ]:
def vc(a_ser):
    '''Return value_counts().to_frame() for a pandas series'''
    
    return a_ser.value_counts().to_frame()

## FILEPATHS

In [ ]:
filepath_courses = "../BILL_PROJ2_FILES/BNCollegeCourses_2022-02-05.csv"

filepath_textbooks = "../BILL_PROJ2_FILES/BNTextbook_2022-02-05.csv"

## PARTITION LARGE TEXTBOOKS CSV to < 100 MB

In [ ]:
data = pd.read_csv(filepath_textbooks, encoding="utf-8", low_memory=False)
 
num_csvs = 4
row_size = 250_000
csvs_list = []
 
for i in range(num_csvs):
    
    df = data[row_size*i:row_size*(i+1)]
    df.to_csv(f"textbooks_{i+1}.csv", encoding="utf-8")
    csvs_list.append(f"textbooks_{i+1}.csv")    

In [ ]:
df_list = []

for filename in csvs_list:
    
    df = pd.read_csv(filename, encoding="utf-8", low_memory=False)
    df_list.append(df)

combo_df = pd.concat(df_list, axis=0)

### Extract CSV into a DataFrame(s)

In [ ]:
textbooks_df = combo_df.copy()

In [ ]:
textbooks_df = basic_clean(textbooks_df)

In [ ]:
courses_df = pd.read_csv(filepath_courses, encoding="Utf-8", low_memory=False)

In [ ]:
courses_df = basic_clean(courses_df)

### Make Changes to the DataFrame(s)

## COURSES

In [ ]:
courses_df = courses_df.drop(["store_id", "catalog_id", "scanDate"], axis=1)

In [ ]:
courses_df = all_caps(courses_df, ["university", "campus", "term", "department", "course", "course_id", "section"])

In [ ]:
vc(courses_df["term"])

### SPRING COURSES

In [ ]:
spring_courses_df = courses_df.copy()

In [ ]:
spring_mask = spring_courses_df["term"].str.contains("SPRING")

In [ ]:
spring_courses_df.loc[spring_mask, "term"] = "SPRING"

In [ ]:
spring_courses_df["term"] = "SPRING"

In [ ]:
vc(spring_courses_df["term"])

In [ ]:
spring_courses_df.to_csv("Resources/spring_courses.csv", encoding="utf-8")

### FALL COURSES

In [ ]:
fall_courses_df = courses_df.copy()

In [ ]:
fall_mask = fall_courses_df["term"].str.contains("FALL")

In [ ]:
fall_courses_df.loc[fall_mask, "term"] = "FALL"

In [ ]:
fall_courses_df["term"] = "FALL"

In [ ]:
vc(fall_courses_df["term"])

In [ ]:
fall_courses_df.to_csv("Resources/fall_courses.csv", encoding="utf-8")

### ALL COURSES

In [ ]:
all_courses_df = pd.concat([fall_courses_df, spring_courses_df], axis=0)

In [ ]:
all_courses_df.sample(50)

In [ ]:
# all_courses_df.to_csv("courses.csv", encoding="utf-8")

# comes in > 100 MB 

## TEXTBOOKS

In [ ]:
textbooks_df = textbooks_df.drop(columns=["catalog_id", "scanDate", "book_url", "no_textbook_message"])

In [ ]:
all_caps(textbooks_df, ["title", "edition", "publisher", "book_type", "book_id", "recommend_type", "price"])

In [ ]:
textbooks_df = textbooks_df.dropna(subset=["title", "ISBN"], axis=0, how="any")

### ISBN column cleanup

In [ ]:
textbooks_df = textbooks_df.astype({"ISBN": "str"}) 

In [ ]:
textbooks_df["ISBN"] = textbooks_df["ISBN"].str.strip(".0")

In [ ]:
vc(textbooks_df["ISBN"])

### price column cleanup

In [ ]:
textbooks_df["price"].info()

In [ ]:
textbooks_df["price"] = textbooks_df["price"].str.lstrip("$")

In [ ]:
textbooks_df["price"] = textbooks_df["price"].str.replace(",", "", regex=False)

In [ ]:
no_price_mask = textbooks_df["price"].str.isupper()

In [ ]:
textbooks_df.loc[no_price_mask, "price"] = "0.00"

In [ ]:
textbooks_df["price"] = textbooks_df["price"].apply(lambda x: float(x))

In [ ]:
vc(textbooks_df["price"])

### book_type column cleanup

In [ ]:
new_textbooks_df = textbooks_df.copy()

digital_textbooks_df = textbooks_df.copy()

In [ ]:
new_mask = new_textbooks_df["book_type"].str.contains("NEW")

In [ ]:
digital_mask = digital_textbooks_df["book_type"].str.contains("DIGITAL")

In [ ]:
new_textbooks_df = new_textbooks_df.loc[new_mask, :]

In [ ]:
vc(new_textbooks_df["book_type"])

In [ ]:
new_textbooks_df["book_type"] = new_textbooks_df["book_type"].replace(["NEW PRINT", "NEW PRINT RENTAL", "BUY NEW", "RENT NEW"], "NEW")

In [ ]:
vc(new_textbooks_df.book_type)

In [ ]:
digital_textbooks_df = digital_textbooks_df.loc[digital_mask, :]

In [ ]:
vc(digital_textbooks_df["book_type"])

In [ ]:
digital_textbooks_df["book_type"] = digital_textbooks_df["book_type"].replace\
    (["DIGITAL PURCHASE", "DIGITAL RENTAL", "BUY DIGITAL", "RENT DIGITAL"], "DIGITAL")
    

In [ ]:
vc(digital_textbooks_df["book_type"])

In [ ]:
new_textbooks_df.to_csv("Resources/new_textbooks.csv", encoding="utf-8")

In [ ]:
digital_textbooks_df.to_csv("Resources/digital_textbooks.csv", encoding="utf-8")

### Create database connection

In [ ]:
protocol = 'postgresql'
host = 'localhost'
database_name = 'postgres_db'
rds_connection_string = f'{protocol}://{user}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

### Load DataFrame(s) into database

In [ ]:
db.to_sql(name='books', con=engine, if_exists='append', index=True)